In [1]:
import requests
import pandas as pd
import geopandas as gp
import basal_and_bark
from basal_and_bark import basal_and_bark as basal
import ipyleaflet
from ipyleaflet import Marker, Popup, WidgetControl, GeoData
import ipywidgets
from ipywidgets import HTML, widgets

In [2]:
leaflet_map = basal.Map(center = [40,-100], zoom = 4, test = "test", scroll_wheel_zoom = True)

       ESTIMATE                       GRP1  \
0  2.299593e+07           `0001 Timberland   
1  9.221615e+04  `0002 Reserved Forestland   
2  5.779893e+03     `0003 Other forestland   
3  9.187540e+06            `0004 Nonforest   
4  1.599913e+05     `0005 Non-Census water   
5  1.107255e+06         `0006 Census water   

                                   GRP2  PLOT_COUNT             SE  \
0                      `0001 Timberland        4258  116168.555463   
1  `0003 Reserved productive forestland          17   22583.437728   
2                `0002 Other forestland           1    5889.315697   
3                       `0005 Nonforest        2144  127095.482790   
4                `0006 Non-Census water          72   24268.138198   
5                    `0007 Census water         205   66828.965605   

   SE_PERCENT      VARIANCE  
0    0.505170  1.349513e+10  
1   24.489678  5.100117e+08  
2  101.893160  3.468404e+07  
3    1.383346  1.615326e+10  
4   15.168414  5.889425e+08  
5    6

       ESTIMATE                       GRP1  \
0  1.267198e+07           `0001 Timberland   
1  1.628825e+05  `0002 Reserved Forestland   
2  1.431640e+04     `0003 Other forestland   
3  6.397835e+06            `0004 Nonforest   
4  1.062155e+05     `0005 Non-Census water   
5  1.139907e+06         `0006 Census water   

                                   GRP2  PLOT_COUNT             SE  \
0                      `0001 Timberland        2643   98390.205282   
1  `0003 Reserved productive forestland          38   29257.554692   
2                `0002 Other forestland           4    7980.836292   
3                       `0005 Nonforest        1507  112648.481505   
4                `0006 Non-Census water          48   19993.299845   
5                    `0007 Census water         202   74232.442218   

   SE_PERCENT      VARIANCE  
0    0.776439  9.680632e+09  
1   17.962364  8.560045e+08  
2   55.746118  6.369375e+07  
3    1.760728  1.268968e+10  
4   18.823341  3.997320e+08  
5    6

In [3]:
leaflet_map.add_layer_control()

In [4]:
leaflet_map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
leaflet_map.add_shp("https://github.com/ZachDorm/basal_and_bark/raw/main/docs/examples/data/tl_2022_us_state.zip")

In [5]:
tn_counties_gd = leaflet_map.addRefData()


In [6]:
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_widget.clear_output()
basemap_ctrl = WidgetControl(widget=output_widget, position='bottomright')
leaflet_map.add_control(basemap_ctrl)

dropdown = widgets.Dropdown(
            options = ["2020", "2021", "2022"], 
            value="2020",
            description='Year',
            )

with output_widget:
            display(dropdown)

In [7]:
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        output_widget.clear_output()
        #leaflet_map.add_layer(Marker(location=kwargs.get('coordinates')))
        latlon = kwargs.get('coordinates')

        gdf = leaflet_map.makePointsFromClick(latlon)

        for i in range(1,len(tn_counties_gd['geometry']),1):
            test = tn_counties_gd['geometry'][i].contains(gdf['geometry'])

            county = 'name'

            if test[0]:
                county = tn_counties_gd['STUSPS'][i]
                with output_widget:
                    display(dropdown)
                    print(county)
                    print(type(county))
                print(leaflet_map.getAPIdata(county, dropdown.value))

        # with output_widget:
        #     print()
            
leaflet_map.on_interaction(handle_click)

In [ ]:
output_widget.widgets[len(output_widget.widgets)-1]

In [ ]:
output_widget.widgets.values()[len(output_widget.widgets.values())-1]

In [ ]:
keys = output_widget.widgets.keys()
keys[len(keys)-1]

In [ ]:
output_widget.widgets["8561c5197658434691ce5b727c699ad0"].value